# Partie 4 : analyse de logs apache

Ici nous allons maintenant utiliser les logs pour répondre à quelques questions :

- quelles sont les causes d'erreurs ?  
- quelles requêtes génèrent-elle le plus de données ?

Nous travaillerons toujours sur le même jeu de données 

In [ ]:
import pandas as pd
import numpy as np
import time

## Le chargement du fichier proprement dit, on indique que les colonnes sont 
## séparées par des ;
apache_log_df = pd.read_csv( "log_aggreges.txt.csv", sep =";" )

## Une fonction pour convertir la colonne "apache_date" en date manipulable par python
apache_log_df["date"] = apache_log_df["apache_date"].apply( 
    lambda apache_date_format: time.strptime(apache_date_format, "%d/%b/%Y:%H:%M:%S") 
     )

## Une fonction pour convertir la colonne "apache_date" en date manipulable par python
apache_log_df["data_size"] = apache_log_df["data_size"].replace("-","0").astype(np.int64)

## Une fonction pour extraire l'heure
apache_log_df["heure"] = apache_log_df["date"].apply( lambda x:x.tm_hour )
apache_log_df["date_heure"] = apache_log_df["date"].apply( lambda x:(x.tm_mday, x.tm_hour ) )

## On peut maintenant avoir un aperçu de ce tableau, avec la fonction head qui permet
## de renvoyer les n premières lignes.
apache_log_df.head(5)

In [ ]:
X = apache_log_df[["request"]]
Xd = X.T.to_dict().values()

from sklearn.feature_extraction import DictVectorizer
prep = DictVectorizer()
Xt = prep.fit_transform(Xd).toarray()

In [ ]:
Xt.shape

In [ ]:
Xt = Xt[:,:-1]
Xt.shape

In [ ]:
result_to_predict = apache_log_df["return_code"] >= 500
result_to_predict.shape

In [ ]:
from sklearn import datasets, linear_model

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(Xt, result_to_predict)

In [ ]:
print( len(regr.coef_) )
print( regr.coef_ )
print( regr.intercept_ )

In [ ]:
print( len(prep.feature_names_) )

In [ ]:
for it_coef, it_feat_name in zip(regr.coef_, prep.feature_names_ ):
    print("La requête", it_feat_name, "a {0:.2f} % de chance de provoquer une erreur".format(it_coef*100) )

In [ ]:
from sklearn.feature_selection import f_regression

from sklearn import datasets, linear_model

# Create linear regression object
regr_fs = f_regression(Xt, result_to_predict)
regr_fs

In [ ]:
import numpy as np

x = np.random.normal( size = 1000 )
y = np.random.normal( size = 1000 )
z = np.random.normal( size = 1000 )

to_pred = 2*x + z

from sklearn.feature_selection import f_regression

from sklearn import datasets, linear_model
obs = n

# Create linear regression object
regr_fs = f_regression(y.reshape(-1,1), to_pred)
regr_fs